• Multiple datasets contain information about movies, we will merge those data sets by some common
columns(primary key, eg:imdb id), and remove duplicates.

• Remove duplicates and redundancy in the Peoeple dataset.

• No imdb id in the Oscar database:

(a) Use year and movie name as the merging key to attach the Imdb id from the Movie table.

(b) Remove any invalid data.

(c) Create a boolean attribute in the Movie table to indicate whether the movie has any kind of
Oscar nomination or not.

• Using Python to scrape the movie descriptions from IMDb.

• Using Python to scrape the URL of people’s profile photos from IMDb, and create an additional
column Photo in the People table to store the URL

In [ ]:
import pandas as pd
import requests
import gc

In [ ]:
#Multiple datasets contain information about movies, 
#we will merge those data sets by some common columns(primary key, eg:imdb id), 
#and remove duplicates.
imdb_movies = pd.read_csv('IMDB movies.csv')

In [ ]:
# No longer use rating table
# imdb_movie_rating = pd.read_csv('IMDB ratings.csv')

In [ ]:
# import movie_genre table
movie_genre = pd.read_csv('MovieGenre.csv', encoding='latin-1')

In [ ]:
print(imdb_movies.columns)
imdb_movies.head()

In [ ]:
print(imdb_movies.info())

In [ ]:
print(movie_genre.columns)
movie_genre.head()

In [ ]:
print(movie_genre.info())

In [ ]:
# Get length of each string in imdbId column
imdbId_lengths = movie_genre['imdbId'].astype(str).str.len()
# Print summary statistics of imdbId_lengths
print(imdbId_lengths.describe())

In [ ]:
# Add leading zeros and convert to string
movie_genre['imdbId'] = movie_genre['imdbId'].apply(lambda x: str(x).zfill(7))

# Add 'tt' to beginning of string
movie_genre['imdbId'] = 'tt' + movie_genre['imdbId']

#print(movie_genre.info())

In [ ]:
#movie_genre.head()

In [ ]:
# Rename imdbId column to Imdb_id
movie_genre.rename(columns={'imdbId': 'imdb_title_id'}, inplace=True)

duplicates = movie_genre.duplicated(subset='imdb_title_id', keep=False)
if any(duplicates):
    print("There are duplicated rows by imdb_name_id in movie_genre.")
else:
    print("There are no duplicated rows by imdb_name_id in movie_genre.")

In [ ]:
movie_genre.drop_duplicates(subset=['imdb_title_id'], inplace=True)
print(movie_genre.info())

In [ ]:
duplicates = imdb_movies.duplicated(subset='imdb_title_id', keep=False)
if any(duplicates):
    print("There are duplicated rows by imdb_name_id in imdb_movies.")
else:
    print("There are no duplicated rows by imdb_name_id in imdb_movies.")

In [ ]:
duplicates = imdb_movies.duplicated(subset=['year','title'], keep=False)
print("Number of duplicates:", duplicates.sum())

In [ ]:
# Perform an inner join on movie_id to combine the tables
movie_data = pd.merge(imdb_movies, movie_genre[['imdb_title_id', 'Imdb Link', 'IMDB Score', 'Poster']], 
                                               on='imdb_title_id')

print(movie_data.shape)

# Filter out tuples with missing or invalid poster URLs
movie_data = movie_data[movie_data['Poster'].notna() & (movie_data['Poster'] != '')]

# Print the resulting dataframe
# movie_data.head()

print(movie_data.shape)

In [ ]:
# Filter out tuples with an IMDb score less than 6.5
movie_data = movie_data.drop(movie_data[movie_data['IMDB Score'] < 6.5].index)

movie_data.shape

In [ ]:
movie_data.head()

In [ ]:
# print('imdb_movies_new shape:', imdb_movies_new.shape)
# print('imdb_movie_rating shape:', imdb_movie_rating.shape)
# print('imdb_movies_new columns:', imdb_movies_new.columns)
# print('imdb_movie_rating columns:', imdb_movie_rating.columns)

In [ ]:
# imdb_movies_new = imdb_movies_new.merge(imdb_movie_rating, 
#                      left_on='imdb_title_id', right_on='imdb_title_id', how='left')
# imdb_movies_new.head()
# Print the shape and column names of the merged dataframe
# print('imdb_movies_new after merge:', imdb_movies_new.shape)
# print('imdb_movies_new columns after merge:', imdb_movies_new.columns)

In [ ]:
#drop columns
print('movie_data columns:', movie_data.columns)

In [ ]:
# Select the columns to keep and rename 'Poster' to 'poster_url'
movie_data_final = movie_data.loc[:, ['imdb_title_id', 'title', 'original_title', 'year', 'genre', 'duration',
                                'country', 'language', 'director', 'description', 'avg_vote', 'votes', 'Poster']]
movie_data_final = movie_data_final.rename(columns={'Poster': 'poster_url'})
print('movie_data_final columns:', movie_data_final.columns)

In [ ]:
print(movie_data['title'].isna().sum())

In [ ]:
#No imdb id in the Oscar database:
#(a) Use year and movie name as the merging key to attach the Imdb id from the Movie table.
#(b) Remove any invalid data.
#(c) Create a boolean attribute in the Movie table to indicate whether the movie has any kind of Oscar nomination or not.
oscar = pd.read_csv('the_oscar_award.csv')
oscar.shape

In [ ]:
oscar.head()

In [ ]:
print(oscar.info())

In [ ]:
oscar.dropna(subset=['film'], how='any', inplace=True)
oscar.shape

In [ ]:
duplicates = imdb_movies.duplicated(subset='imdb_title_id', keep=False)
if any(duplicates):
    print("There are duplicated rows by imdb_name_id in imdb_movies.")
else:
    print("There are no duplicated rows by imdb_name_id in imdb_movies.")

In [ ]:
oscar['film'] = oscar['film'].str.lower().str.strip()
imdb_movies_title = movie_data.loc[:, ['year','original_title','imdb_title_id','title']]
imdb_movies_title['original_title'] = imdb_movies_title['original_title'].str.lower().str.strip()
imdb_movies_title['title'] = imdb_movies_title['title'].str.lower().str.strip()

In [ ]:
# Based on our test, the film in Oscar data could be matched to either title or original_title in imdb_movies data
# So we created a dictionary to merge the imdb_title_id to oscar data
# create a dictionary mapping (year, film) pairs to imdb_title_ids
film_to_title_id = {}

# match 'film' in Oscar dataset to 'title' in IMDB dataset
title_to_id = dict(zip(zip(imdb_movies_title['year'], imdb_movies_title['title']), imdb_movies_title['imdb_title_id']))
film_to_title_id.update(title_to_id)

# match 'film' in Oscar dataset to 'original_title' in IMDB dataset
orig_title_to_id = dict(zip(zip(imdb_movies_title['year'], imdb_movies_title['original_title']), imdb_movies_title['imdb_title_id']))
film_to_title_id.update(orig_title_to_id)

# create a new column in Oscar dataset with imdb_title_id values
oscar['imdb_title_id'] = oscar[['year_film', 'film']].apply(lambda x: film_to_title_id.get(tuple(x)), axis=1)

# delete oscar movies where imdb_title_id is missing
oscar.dropna(subset=['imdb_title_id'], inplace=True)

In [ ]:
print(oscar.shape)
print(oscar.columns)

In [ ]:
oscar_data = oscar.merge(movie_data[['imdb_title_id', 'original_title']], on='imdb_title_id', how='left')

# Rename 'title' column to 'movie_title'
oscar_data = oscar_data.rename(columns={'original_title': 'movie_title'})

# Drop 'film' column
oscar_data = oscar_data.drop(columns=['film'])

oscar_data.columns

In [ ]:
duplicates = oscar_data.duplicated(subset=['ceremony','category','name','movie_title'], keep=False)
print("Number of duplicates:", duplicates.sum())

In [ ]:
duplicates = oscar_data.duplicated(subset=['ceremony', 'category', 'name', 'movie_title'], keep=False)
oscar_data.drop(oscar_data.loc[duplicates & ~oscar_data['winner']].index, inplace=True)
duplicates = oscar_data.duplicated(subset=['ceremony', 'category', 'name', 'movie_title'], keep=False)
print(oscar_data[duplicates])

In [ ]:
# export the data
oscar_data.to_csv('oscar_data_test.csv', index=False)

In [ ]:
# Create a boolean column indicating if the imdb_title_id is in oscar_data
movie_data_final['Oscar_nominated'] = movie_data_final['imdb_title_id'].isin(oscar_data['imdb_title_id'])

In [ ]:
movie_data_final.columns

In [ ]:
movie_data_final.shape

In [ ]:
# export the data
# movie_data_final.to_csv('movie_data_final.csv', index=False)

In [ ]:
gc.collect()

In [ ]:
#Remove duplicates and redundancy in the Peoeple dataset.
imdb_names = pd.read_csv('IMDB names.csv')

In [ ]:
print(imdb_names.columns)

In [ ]:
duplicates = imdb_names.duplicated(subset='imdb_name_id', keep=False)

if any(duplicates):
    print("There are duplicated rows by imdb_name_id.")
else:
    print("There are no duplicated rows by imdb_name_id.")

In [ ]:
# Drop unnecessary columncs
imdb_names = imdb_names.loc[:, ['imdb_name_id', 'name', 'birth_name', 
                                       'date_of_birth', 'place_of_birth', 
                                       'date_of_death', 'height', 'bio']]

In [ ]:
movie_people = pd.read_csv('IMDb title_principals.csv')

In [ ]:
movie_people.head()

In [ ]:
print(movie_people.info())

In [ ]:
# Perform an inner join on movie_id to combine the tables
staff_data = pd.merge(movie_data['imdb_title_id'], movie_people, on='imdb_title_id')

# Filter the resulting dataframe to include only the rows where the category is "actor"
#actors = merged_data[merged_data['category'] == 'actor']
staff_data.info()

In [ ]:
duplicates = staff_data.duplicated(subset=['imdb_title_id', 'ordering'], keep=False)

In [ ]:
staff_data = staff_data.loc[:, ['imdb_title_id', 'ordering', 'imdb_name_id', 
                                       'category']]

staff_data.columns

In [ ]:
# export the data
# staff_data.to_csv('movie_people.csv', index=False)

In [ ]:
movie_people_df = pd.read_csv('movie_people.csv')

In [ ]:
category_counts_df = movie_people_df.groupby("imdb_name_id")["category"].value_counts().unstack(fill_value=0)

# Compute the total number of rows for each person
total_counts_df = movie_people_df.groupby("imdb_name_id").size().to_frame(name="num_all")

# Combine the category counts and total counts into a single DataFrame
counts_df = pd.concat([category_counts_df, total_counts_df], axis=1)

# Compute the sum of the num_actor, num_actress, and num_director columns
if "num_actor" in counts_df.columns and "num_actress" in counts_df.columns and "num_director" in counts_df.columns:
    counts_df["num_aad"] = counts_df["num_actor"] + counts_df["num_actress"] + counts_df["num_director"]
else:
    counts_df["num_aad"] = 0

# Sort the DataFrame by the num_aad column in descending order
counts_df = counts_df.sort_values("num_aad", ascending=False)

# Rename the columns to include the "num_" prefix
counts_df = counts_df.add_prefix("num_")

# Reset the index to include the "imdb_name_id" column
counts_df = counts_df.reset_index()

# Print the resulting DataFrame
print(counts_df)

In [ ]:
# Get the number of unique people in the merged dataframe
unique_people = staff_data['imdb_name_id'].nunique()

# Print the number of unique people
print("Number of unique people after merge:", unique_people)

In [ ]:
# Get the imdb_name_ids that exist in the staff_data DataFrame
valid_ids = staff_data['imdb_name_id'].unique()

# Filter the imdb_names DataFrame by checking if the imdb_name_id column is in valid_ids
imdb_names_new = imdb_names[imdb_names['imdb_name_id'].isin(valid_ids)]

In [ ]:
imdb_names_new.shape

In [ ]:
imdb_names_new.columns

In [ ]:
# imdb_names_new.to_csv('people.csv', index=False)

In [ ]:
# Using Python to scrape the URL of people’s profile photos from IMDb, and create an additional column Photo in the People table to store the URL

In [ ]:
pip install IMDbPY


In [ ]:
from imdb import IMDb
import concurrent.futures
import csv
import socket
import pandas as pd
import requests
import gc

import numpy as np
import time

In [ ]:
imdb_names_new = pd.read_csv('people.csv')

In [ ]:

#imdb_names_new = pd.read_csv('people.csv')

#imdb_names_new = imdb_names_new.iloc[:1000]

# Divide the DataFrame into 10 equal parts
parts = np.array_split(imdb_names_new, 100)

# Define a function to fetch the headshot for a single IMDb id and write the photo URL to a CSV file
def get_headshot(imdb_id):
    try:
        person = imdb.get_person(imdb_id)
        photo_url = person.get("headshot", None)
        return photo_url
    except Exception as e:
        print(f"Error fetching headshot for IMDb id {imdb_id}: {e}")
        return None

# Fetch the headshots for all IMDb ids in each part using parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, part in enumerate(parts):
        imdb_ids = [int(imdb_name_id[2:]) for imdb_name_id in part["imdb_name_id"]]
        futures = [executor.submit(get_headshot, imdb_id) for imdb_id in imdb_ids]
        photo_urls = [future.result() for future in futures]
        with open(f"photo_urls_{i}.csv", mode="w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            try:
                for imdb_id, photo_url in zip(imdb_ids, photo_urls):
                    writer.writerow([imdb_id, photo_url])
            finally:
                f.close()
        # Print the time finished for the part
        print(f"Part {i} finished at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        # Clear the memory
        del imdb_ids, futures, photo_urls
        gc.collect()


In [ ]:
# Read in all CSV files and stack them together
df_list = []
for i in range(100):
    file_name = f"photo_urls_{i}.csv"
    df = pd.read_csv(file_name, header=None, names=["imdb_name_id_int", "photo_url"])
    df_list.append(df)

people_urls = pd.concat(df_list, ignore_index=True)

# Add leading zeros to imdb_name_id_int and prefix with "nm"
people_urls["imdb_name_id"] = "nm" + people_urls["imdb_name_id_int"].apply(lambda x: str(x).zfill(7))


In [ ]:
people_urls.head()

In [ ]:
#remove the format and size suffix of urls 

import re

people_urls['photo_url'] = people_urls['photo_url'].str.replace('\._V1_.+', '', regex=True).str.replace('\._V1_', '', regex=True)

In [ ]:
imdb_names_final = imdb_names_new.merge(people_urls[['imdb_name_id','photo_url']], 
                     left_on='imdb_name_id', right_on='imdb_name_id', how='left')

In [ ]:
imdb_names_final = pd.read_csv('people_test.csv')
max_len = imdb_names_final["bio"].str.len().max()
print(max_len)

In [ ]:
print(imdb_names_final.shape[0])

In [ ]:
length_counts = imdb_names_final["bio"].str.len().value_counts()

# Print the resulting distribution
print(length_counts.sort_index())

In [ ]:
imdb_names_final.to_csv('people_test.csv', index=False)

In [ ]:
# Get length of each string in imdbId column
lengths = people_urls['photo_url'].astype(str).str.len()
# Print summary statistics of imdbId_lengths
print(lengths.describe())

In [ ]:
imdb_movie_forposter = pd.read_csv('movie_data_final.csv')

In [ ]:
print(len(imdb_movie_forposter))

In [ ]:
from imdb import IMDb

parts = np.array_split(imdb_movie_forposter,100)

# Define a function to fetch the poster for a single IMDb id and write the photo URL to a CSV file
def get_poster(imdb_id):
    try:
        imdb = IMDb()
        movie = imdb.get_movie(imdb_id)
        poster_url = movie.get('full-size cover url')
        return poster_url
    except Exception as e:
        print(f"Error fetching poster for IMDb id {imdb_id}: {e}")
        return None

# Fetch the poster for all IMDb ids in each part using parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, part in enumerate(parts):
        if i < 83:
            continue
        imdb_ids = [int(imdb_title_id[2:]) for imdb_title_id in part["imdb_title_id"]]
        futures = [executor.submit(get_poster, imdb_id) for imdb_id in imdb_ids]
        poster_urls = [future.result() for future in futures]
        with open(f"movie_poster_urls_{i}.csv", mode="w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            try:
                for imdb_id, poster_url in zip(imdb_ids, poster_urls):
                    writer.writerow([imdb_id, poster_url])
            finally:
                f.close()
        # Print the time finished for the part
        print(f"Part {i} finished at {time.strftime('%Y-%m-%d %H:%M:%S')}")
        # Clear the memory
        del imdb_ids, futures, poster_urls
        gc.collect()


In [ ]:
# Read in all CSV files and stack them together
df_list = []
for i in range(100):
    file_name = f"movie_poster_urls_{i}.csv"
    df = pd.read_csv(file_name, header=None, names=["imdb_title_id_int", "poster_url"])
    df_list.append(df)

movie_poster_urls = pd.concat(df_list, ignore_index=True)

# Add leading zeros to imdb_name_id_int and prefix with "nm"
movie_poster_urls["imdb_title_id"] = "tt" + movie_poster_urls["imdb_title_id_int"].apply(lambda x: str(x).zfill(7))

In [ ]:
imdb_movie_forposter = imdb_movie_forposter.drop('poster_url', axis=1)
movie_poster_final = imdb_movie_forposter.merge(movie_poster_urls[['imdb_title_id','poster_url']], 
                                                 left_on='imdb_title_id', right_on='imdb_title_id', how='left')


In [ ]:
movie_poster_final = movie_poster_final.rename(columns={"title": "original_title", "original_title": "title"})

In [ ]:
print("Number of rows:", movie_poster_final.shape[0])

In [ ]:
max_len = movie_poster_final["description"].str.len().max()
print(max_len)

In [ ]:
movie_poster_final2 = movie_poster_final.dropna()

In [ ]:
print("Number of rows:", movie_poster_final2.shape[0])

In [ ]:
# movie_poster_final.to_csv('movie_test.csv', index=False)
movie_poster_final2.to_csv('movie_test2.csv', index=False)

In [ ]:
# list all genres with frequencies sorted by frequency
genres = {}
for genre_list in movie_poster_final["genre"].str.split(", "):
    for genre in genre_list:
        if genre in genres:
            genres[genre] += 1
        else:
            genres[genre] = 1
genres_sorted = sorted(genres.items(), key=lambda x: x[1], reverse=True)
print("Genres: ", genres_sorted)

# list all languages with frequencies sorted by frequency
languages = {}
for language_list in movie_poster_final["language"]:
    if isinstance(language_list, str):
        for language in language_list.split(", "):
            if language in languages:
                languages[language] += 1
            else:
                languages[language] = 1
languages_sorted = sorted(languages.items(), key=lambda x: x[1], reverse=True)
print("Languages: ", languages_sorted)

# list all countries with frequencies sorted by frequency
countries = {}
for country_list in movie_poster_final["country"].str.split(", "):
    for country in country_list:
        if country in countries:
            countries[country] += 1
        else:
            countries[country] = 1
countries_sorted = sorted(countries.items(), key=lambda x: x[1], reverse=True)
print("Countries: ", countries_sorted)


In [ ]:
movie_poster_final2

In [ ]:
import pandas as pd
import requests
import gc
import matplotlib.pyplot as plt

In [ ]:
movie_data = pd.read_csv('movie_test2.csv')
print(movie_data.shape)

In [ ]:
for column in movie_data.columns:
    if movie_data[column].dtype == 'object':
        max_len = movie_data[column].str.len().max()
        print(f"The maximum length of '{column}' is {max_len}")
    else:
        print(f"'{column}' is not a string column")

In [ ]:
oscar_data = pd.read_csv('oscar_data_test.csv')
print(oscar_data.shape)

In [ ]:
people_data = pd.read_csv('people_test.csv')
print(people_data.shape)

In [ ]:
people_data = people_data[people_data['name'].notnull() & (people_data['name'] != '')]
print(people_data.shape)

In [ ]:
movie_people = pd.read_csv('movie_people.csv')
print(movie_people.shape)

In [ ]:
movie_people_new = movie_people[movie_people['imdb_title_id'].isin(movie_data['imdb_title_id']) &
                                movie_people['imdb_name_id'].isin(people_data['imdb_name_id'])]
print(movie_people_new.shape)

In [ ]:
people_data_new = people_data[people_data['imdb_name_id'].isin(movie_people_new['imdb_name_id'])]
print(people_data_new.shape)

In [ ]:
oscar_data_new = oscar_data[oscar_data['imdb_title_id'].isin(movie_data['imdb_title_id'])]
print(oscar_data_new.shape)                       

In [ ]:
movie_data['Oscar_nominated'] = movie_data['imdb_title_id'].isin(oscar_data_new['imdb_title_id'])

In [ ]:
def convert_date(date_str):
    try:
        # try parsing the date as mm/dd/yy format
        return pd.to_datetime(date_str, format='%m/%d/%y').strftime('%m/%d/%Y')
    except:
        try:
            # try parsing the date as mm/dd/yyyy format
            return pd.to_datetime(date_str, format='%m/%d/%Y').strftime('%m/%d/%Y')
        except:
            try:
                # try parsing the date as yyyy-mm-dd format
                return pd.to_datetime(date_str, format='%Y-%m-%d').strftime('%m/%d/%Y')
            except:
                try:
                    # try extracting the year from the date string
                    year = re.search('\d{4}', date_str).group(0)
                    return f'01/01/{year}'
                except:
                    # for all other cases, return None
                    return None


In [ ]:
people_data_new['date_of_birth'] = people_data_new['date_of_birth'].apply(convert_date)
people_data_new['date_of_death'] = people_data_new['date_of_death'].apply(convert_date)

In [ ]:
max_len = people_data_new['date_of_birth'].str.len().max()
print(max_len)

In [ ]:
movie_data.to_csv('final/movie_data.csv', index=False)

In [ ]:
movie_people_new.to_csv('final/movie_people.csv', index=False)

In [ ]:
people_data_new.to_csv('final/people_data.csv', index=False)

In [ ]:
oscar_data_new.to_csv('final/oscar_data.csv', index=False)